# All prompt examples

In [ ]:
# ruff: noqa: T201, T203
from __future__ import annotations

import os
from pathlib import Path

if Path.cwd().name == "notebooks":
    os.chdir("..")

import logging

from dotenv import load_dotenv

from src.formatting import format_oracle_pairs_filepath, format_subsets_ontologies_paths
from src.onto_access import OntologyAccess
from src.onto_object import OntologyEntryAttr
from src.prompts.prompt_utils import format_hierarchy
from src.prompts.prompts import (
    prompt_direct_entity,
    prompt_direct_entity_ontological,
    prompt_direct_entity_with_synonyms,
    prompt_sequential_hierarchy,
    prompt_sequential_hierarchy_ontological,
    prompt_sequential_hierarchy_with_synonyms,
)
from src.utils import read_oracle_pairs

logging.getLogger().setLevel(logging.WARNING)
load_dotenv()

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
DATASET_NAME, SET_NAME = "anatomy", "human-mouse"

src_onto_path, tgt_onto_path = format_subsets_ontologies_paths(DATASET_NAME, SET_NAME)

onto_src = OntologyAccess(src_onto_path, annotate_on_init=True)
onto_tgt = OntologyAccess(tgt_onto_path, annotate_on_init=True)

* Owlready2 * Creating new ontology human <data/anatomy/human-mouse/human.owl#>.
* Owlready2 * ADD TRIPLE data/anatomy/human-mouse/human.owl http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#Ontology
* Owlready2 *     ...loading ontology human from data/anatomy/human-mouse/human.owl...
* Owlready2 *     ...8 properties found: ObsoleteProperty, UNDEFINED_part_of, label, hasRelatedSynonym, hasDefaultNamespace, savedBy, hasDate, hasDefinition


* Owlready2 * Reseting property oboInOwl.ObsoleteProperty: new triples are now available.
* Owlready2 * Reseting property oboInOwl.hasRelatedSynonym: new triples are now available.
* Owlready2 * Reseting property oboInOwl.hasDefaultNamespace: new triples are now available.
* Owlready2 * Reseting property oboInOwl.savedBy: new triples are now available.
* Owlready2 * Reseting property oboInOwl.hasDate: new triples are now available.


In [91]:
src_entity, tgt_entity = read_oracle_pairs(format_oracle_pairs_filepath(DATASET_NAME, SET_NAME))[41] # consider testing: 34, 39, 41
try:
    src_entity, tgt_entity = OntologyEntryAttr(src_entity, onto_src), OntologyEntryAttr(tgt_entity, onto_tgt)
except AssertionError:
    src_entity, tgt_entity = OntologyEntryAttr(tgt_entity, onto_src), OntologyEntryAttr(src_entity, onto_tgt)

# New prompts instances

### Ontological prompts

In [102]:
print(prompt_direct_entity_ontological(src_entity, tgt_entity))

Analyze the following entities, each originating from a distinct biomedical ontology.
Your task is to assess whether they represent the **same ontological concept**, considering both their semantic meaning and hierarchical position.

1. Source entity: "cuboid"
	- Direct ontological parent: tarsal bone

2. Target entity: "Cuboid_Bone"
	- Direct ontological parent: Tarsal_Bone

Are these entities **ontologically equivalent** within their respective ontologies? Respond with "True" or "False".


In [103]:
print(prompt_sequential_hierarchy_ontological(src_entity, tgt_entity))

Analyze the following entities, each originating from a distinct biomedical ontology.
Each is represented by its **ontological lineage**, capturing its hierarchical placement from the most general to the most specific level.

1. Source entity ontological lineage:
	Level 0: cuboid
	Level 1: tarsal bone
	Level 2: carpal/tarsal bone, foot bone

2. Target entity ontological lineage:
	Level 0: Cuboid_Bone
	Level 1: Tarsal_Bone
	Level 2: Bone_of_the_Lower_Extremity

Based on their **ontological positioning, hierarchical relationships, and semantic alignment**, do these entities represent the **same ontological concept**? Respond with "True" or "False".


### Natural language prompts

In [100]:
print(prompt_direct_entity(src_entity, tgt_entity))

We have two entities from different biomedical ontologies.
The first one is "cuboid", which belongs to the broader category "tarsal bone"
The second one is "Cuboid_Bone", which belongs to the broader category "Tarsal_Bone"

Do they mean the same thing? Respond with "True" or "False".


In [101]:
print(prompt_sequential_hierarchy(src_entity, tgt_entity))

We have two entities from different biomedical ontologies.
The first one is "cuboid", which belongs to the broader category "tarsal bone", under the even broader category "carpal/tarsal bone, foot bone"
The second one is "Cuboid_Bone", which belongs to the broader category "Tarsal_Bone", under the even broader category "Bone_of_the_Lower_Extremity"

Do they mean the same thing? Respond with "True" or "False".


### Natural language prompts with synonyms

In [98]:
print(prompt_direct_entity_with_synonyms(src_entity, tgt_entity))

We have two entities from different biomedical ontologies.
The first one is "cuboid", also known as "foot distal carpal bone 4 and 5", which falls under the category "tarsal bone".
The second one is "Cuboid_Bone", also known as "Cuboid Bone", which falls under the category "Tarsal_Bone".

Do they mean the same thing? Respond with "True" or "False".


In [99]:
print(prompt_sequential_hierarchy_with_synonyms(src_entity, tgt_entity))

We have two entities from different biomedical ontologies.
The first one is "cuboid", also known as "foot distal carpal bone 4 and 5", belongs to broader category "tarsal bone", under the even broader category "carpal/tarsal bone, foot bone".
The second one is "Cuboid_Bone", also known as "Cuboid Bone", belongs to broader category "Tarsal_Bone" (also known as "Tarsal Bones", "Tarsal Bone"), under the even broader category "Bone_of_the_Lower_Extremity" (also known as "Bone of the Lower Extremity", "Bones of Lower Extremity", "Bone of Lower Extremity").

Do they mean the same thing? Respond with "True" or "False".


### Get full info

In [71]:
print(format_hierarchy(src_entity.get_parents_by_levels(max_level=3), False, True))

	Level 0: cuboid
	Level 1: tarsal bone
	Level 2: carpal/tarsal bone, foot bone
	Level 3: hindlimb bone


In [72]:
print(format_hierarchy(tgt_entity.get_parents_by_levels(max_level=3), False, True))

	Level 0: Cuboid_Bone
	Level 1: Tarsal_Bone
	Level 2: Bone_of_the_Lower_Extremity
	Level 3: Bone_of_the_Extremity


In [73]:
print(src_entity)
print(src_entity.get_all_entity_names())
print(src_entity.get_parents_preferred_names())
print(src_entity.get_children_preferred_names())
print(src_entity.get_synonyms())
print(src_entity.get_direct_parents())
print(src_entity.get_direct_parents().pop().get_direct_children())

{'class': mouse.MA_0001355, 'uri': 'http://mouse.owl#MA_0001355', 'preffered_names': {'cuboid'}, 'synonyms': {'foot distal carpal bone 4 and 5'}, 'all_names': {'cuboid', 'foot distal carpal bone 4 and 5'}, 'parents': {mouse.MA_0001459, owl.Thing, mouse.MA_0000297, mouse.MA_0000688, mouse.MA_0000660, mouse.MA_0000295, mouse.MA_0000643}, 'children': set()}
{'cuboid', 'foot distal carpal bone 4 and 5'}
[{'Thing'}, {'tarsal bone'}, {'limb bone'}, {'hindlimb bone'}, {'foot bone'}, {'carpal/tarsal bone'}, {'bone'}]
[]
{'foot distal carpal bone 4 and 5'}
{mouse.MA_0000297}
{mouse.MA_0001349, mouse.MA_0001354, mouse.MA_0001352, mouse.MA_0001350, mouse.MA_0001355, mouse.MA_0001348, mouse.MA_0001353, mouse.MA_0001351}


In [74]:
print(tgt_entity)
print(tgt_entity.get_all_entity_names())
print(tgt_entity.get_parents_preferred_names())
print(tgt_entity.get_children_preferred_names())
print(tgt_entity.get_synonyms())
print(tgt_entity.get_direct_parents())
print(tgt_entity.get_direct_parents().pop().get_direct_children())

{'class': human.NCI_C32414, 'uri': 'http://human.owl#NCI_C32414', 'preffered_names': {'Cuboid_Bone'}, 'synonyms': {'Cuboid Bone'}, 'all_names': {'Cuboid Bone', 'Cuboid_Bone'}, 'parents': {human.NCI_C12982, human.NCI_C33904, owl.Thing, human.NCI_C25769, human.NCI_C32221, human.NCI_C12366, human.NCI_C34076, human.NCI_C32223, human.NCI_C12219, human.NCI_C12796}, 'children': set()}
{'Cuboid Bone', 'Cuboid_Bone'}
[{'Bone_of_the_Lower_Extremity'}, {'Other_Anatomic_Concept'}, {'Thing'}, {'Musculoskeletal_System_Part'}, {'Body_Part'}, {'Bone'}, {'Skeletal_System_Part'}, {'Bone_of_the_Extremity'}, {'Anatomic_Structure_System_or_Substance'}, {'Tarsal_Bone'}]
[]
{'Cuboid Bone'}
{human.NCI_C12796}
{human.NCI_C32416, human.NCI_C32250, human.NCI_C52799, human.NCI_C32156, human.NCI_C33162, human.NCI_C32414}
